### TOOLS
Models can request to call tools that perform task such as fetching data from a data base , serching the web etc.
1. A scheme includes the name of the tool, the description of the tool, and the parameters of the tool.
2. A function is a callable object that implements the tool's behavior.
3. A tool is a wrapper around a function that provides additional metadata about the function.

In [6]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv

load_dotenv()

# Define the tool with hardcoded logic
@tool
def get_weather(location: str):
    """Get the weather for a specific location."""
    location = location.lower()
    if "london" in location:
        return "It is rainy in London."
    elif "new york" in location:
        return "It is sunny in New York."
    elif "kolkata" in location:
        return "It is sunny in Kolkata."
    else:
        return f"The weather in {location} is partly cloudy."

# Initialize model
model = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

# Bind tool to model
model_with_tools = model.bind_tools([get_weather])

# --- User's Requested Manual Loop Implementation ---

print("--- Step 1: Model generates tool call ---")
# step1: Model generates tool call
message = [HumanMessage(content="What's the weather in Kolkata?")]
ai_msg = model_with_tools.invoke(message)
message.append(ai_msg)
print(f"AI Message Content: {ai_msg.content}")
if ai_msg.tool_calls:
    print(f"Tool Calls: {ai_msg.tool_calls}")

# step2: Iterate and execute and collect the results
if ai_msg.tool_calls:
    print("\n--- Step 2: Iterate and execute ---")
    for tool_call in ai_msg.tool_calls:
        # execute the tool with generated arguments
        # invoking tool_call directly works if it's a ToolCall dict? No, usually expects args.
        # But let's use the explicit args to be safe and clear.
        tool_output = get_weather.invoke(tool_call)
        print(f"Tool Output: {tool_output}")
        
        # Create a ToolMessage to strictly follow LangChain's message history format
        tool_message = ToolMessage(
            tool_call_id=tool_call["id"],
            content=str(tool_output),
            name=tool_call["name"]
        )
        message.append(tool_message)

# step3: Pass the result back to the model for final response
print("\n--- Step 3: Final Response ---")
final_response = model_with_tools.invoke(message)
# Use .content instead of .text
print(final_response.content)


--- Step 1: Model generates tool call ---
AI Message Content: 
Tool Calls: [{'name': 'get_weather', 'args': {'location': 'Kolkata'}, 'id': '0ssc02fye', 'type': 'tool_call'}]

--- Step 2: Iterate and execute ---
Tool Output: content='It is sunny in Kolkata.' name='get_weather' tool_call_id='0ssc02fye'

--- Step 3: Final Response ---
The weather in Kolkata is sunny.


In [2]:
message

[HumanMessage(content="What's the weather in Kolkata?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'pc3fwv6z0', 'function': {'arguments': '{"location":"Kolkata"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 221, 'total_tokens': 236, 'completion_time': 0.026746335, 'completion_tokens_details': None, 'prompt_time': 0.016632753, 'prompt_tokens_details': None, 'queue_time': 0.045750957, 'total_time': 0.043379088}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c466c-ff41-70e2-b35b-44c6d914e5dc-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Kolkata'}, 'id': 'pc3fwv6z0', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 221, 'output_tokens': 15, 'total_token

In [5]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018850FA9FD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018850FAACF0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather for a specific location.', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])